# Wood Beam Clips

In [1]:
from functools import partial
import numpy as np
import pandas as pd
from wsweng.wood.dowels import wood_bolt

## Givens

In [2]:
diameter = [0.625, 0.75, 0.875, 1.0]  # Nominal diameter, inches
thickness = [3.00, 3.50, 5.125, 5.25, 5.50, 7.0]
clip_thickness = 3/16.0
species = "DFL"
steel = "A36"
make_bolt = partial(
    wood_bolt,
    full_diameter=False,
    main_material=species,
    side_material=steel,
    side_thickness=clip_thickness
)

## Schedule

Calculate Z for each combination of diameter and N

In [3]:
def round_to(x: float, xto: float) -> float:
    return xto*np.ceil(x/xto)

frame_list = []
for dia in diameter:
    edge_distance = round_to(4*dia, 0.25)
    spacing = round_to(4*dia, 0.25)
    for thk in thickness:
        for dbl_v in (False, True):
            z = make_bolt(dia, thk, double_shear=dbl_v).Zv(theta=90.0)
            new_frame = pd.DataFrame(np.arange(2, 6, 1), columns=["N"])
            new_frame["D"] = dia
            new_frame["Thk"] = thk
            new_frame["DblV"] = dbl_v
            new_frame["Height"] = 2*edge_distance + (new_frame["N"] - 1)*spacing
            new_frame["Z"] = z*new_frame["N"]
            frame_list.append(new_frame)

D = pd.concat(frame_list)
D.sample(6)

,N,D,Thk,DblV,Height,Z
3,5,0.625,7.000,True,15.0,4696.656789
3,5,0.875,5.125,False,21.0,4207.789644
2,4,0.750,5.500,True,15.0,5156.307775
3,5,0.625,7.000,False,15.0,2348.328394
0,2,0.625,3.000,False,7.5,842.329827
0,2,1.000,5.125,False,12.0,1809.902874


## Design Schedule

In [4]:
D_sched = D[D["Height"]<=36].copy(deep=True)
D_sched['Z'] = np.round(D_sched["Z"], 1)
D_sched = D_sched.pivot(index=["D", "N", "Height"], columns=["Thk", "DblV"], values="Z")
D_sched

Thk              3.000           3.500           5.125           5.250  \
DblV             False    True   False    True   False    True   False   
D     N Height                                                           
0.625 2 7.5      842.3  1742.0   939.3  1878.7   939.3  1878.7   939.3   
      3 10.0    1263.5  2613.0  1409.0  2818.0  1409.0  2818.0  1409.0   
      4 12.5    1684.7  3483.9  1878.7  3757.3  1878.7  3757.3  1878.7   
      5 15.0    2105.8  4354.9  2348.3  4696.7  2348.3  4696.7  2348.3   
0.750 2 9.0      946.5  1939.8  1081.6  2263.1  1289.1  2578.2  1289.1   
      3 12.0    1419.8  2909.7  1622.4  3394.6  1933.6  3867.2  1933.6   
      4 15.0    1893.1  3879.6  2163.2  4526.2  2578.2  5156.3  2578.2   
      5 18.0    2366.4  4849.5  2704.0  5657.7  3222.7  6445.4  3222.7   
0.875 2 10.5    1041.4  2116.7  1187.7  2469.5  1683.1  3376.2  1688.1   
      3 14.0    1562.1  3175.0  1781.6  3704.2  2524.7  5064.3  2532.2   
      4 17.5    2082.8  4233.4  2375.4  4939.0  3366.2  6752.4  3376.2   
      5 21.0    2603.5  5291.7  2969.3  6173.7  4207.8  8440.5  4220.3   
1.000 2 12.0    1125.0  2269.3  1280.8  2647.6  1809.9  3876.8  1851.5   
      3 16.0    1687.4  3404.0  1921.2  3971.4  2714.9  5815.2  2777.2   
      4 20.0    2249.9  4538.7  2561.6  5295.1  3619.8  7753.6  3702.9   
      5 24.0    2812.4  5673.4  3201.9  6618.9  4524.8  9692.0  4628.6   

Thk                      5.500            7.000           
DblV              True   False     True   False     True  
D     N Height                                            
0.625 2 7.5     1878.7   939.3   1878.7   939.3   1878.7  
      3 10.0    2818.0  1409.0   2818.0  1409.0   2818.0  
      4 12.5    3757.3  1878.7   3757.3  1878.7   3757.3  
      5 15.0    4696.7  2348.3   4696.7  2348.3   4696.7  
0.750 2 9.0     2578.2  1289.1   2578.2  1289.1   2578.2  
      3 12.0    3867.2  1933.6   3867.2  1933.6   3867.2  
      4 15.0    5156.3  2578.2   5156.3  2578.2   5156.3  
      5 18.0    6445.4  3222.7   6445.4  3222.7   6445.4  
0.875 2 10.5    3376.2  1688.1   3376.2  1688.1   3376.2  
      3 14.0    5064.3  2532.2   5064.3  2532.2   5064.3  
      4 17.5    6752.4  3376.2   6752.4  3376.2   6752.4  
      5 21.0    8440.5  4220.3   8440.5  4220.3   8440.5  
1.000 2 12.0    3971.4  1934.8   4160.5  2118.6   4237.2  
      3 16.0    5957.0  2902.2   6240.7  3177.9   6355.8  
      4 20.0    7942.7  3869.6   8320.9  4237.2   8474.4  
      5 24.0    9928.4  4837.0  10401.2  5296.5  10593.0

!jupyter nbconvert --to HTML beam_clips.ipynb

In [36]:
# print(D_sched.loc[0.625:0.750, (slice(None), True)].to_markdown())

# Single Shear - Supported Member

|  Member Width -->|     3" | 3-1/2" | 5-1/8" | 5-1/4" | 5-1/2" |     7" |
|:-----------------|-------:|-------:|-------:|-------:|-------:|-------:|
| (2)-5/8 7.5"     |  842.3 |  939.3 |  939.3 |  939.3 |  939.3 |  939.3 |
| (2)-3/4 9.0"     |  946.5 | 1081.6 | 1289.1 | 1289.1 | 1289.1 | 1289.1 |
| (3)-3/4 12.0"    | 1419.8 | 1622.4 | 1933.6 | 1933.6 | 1933.6 | 1933.6 |
| (4)-3/4 15.0"    | 1893.1 | 2163.2 | 2578.2 | 2578.2 | 2578.2 | 2578.2 |
| (5)-3/4 18.0"    | 2366.4 | 2704   | 3222.7 | 3222.7 | 3222.7 | 3222.7 |

# Double Shear - Supporting Member

|  Member Width -->|     3" | 3-1/2" | 5-1/8" | 5-1/4" | 5-1/2" |     7" |
|:-----------------|-------:|-------:|-------:|-------:|-------:|-------:|
| (2)-5/8 7.5"     | 1742.0 | 1878.7 | 1878.7 | 1878.7 | 1878.7 | 1878.7 |
| (2)-3/4 9.0"     | 1939.8 | 2263.1 | 2578.2 | 2578.2 | 2578.2 | 2578.2 |
| (3)-3/4 12.0"    | 2909.7 | 3394.6 | 3867.2 | 3867.2 | 3867.2 | 3867.2 |
| (4)-3/4 15.0"    | 3879.6 | 4526.2 | 5156.3 | 5156.3 | 5156.3 | 5156.3 |
| (5)-3/4 18.0"    | 4849.5 | 5657.7 | 6445.4 | 6445.4 | 6445.4 | 6445.4 |

Notes
- All members assumed with G=0.50 or better.
- Group action factor assumed to be 1.0.